## Student Homework Sheet — Stage 04: Data
## Acquisition and Ingestion

In [31]:
#API PULL

import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

#pathlib = library utk pilih direktori folder
DATA_RAW = pathlib.Path("data/raw")

#mkdir untuk bikin folder
DATA_RAW.mkdir(parents=True, exist_ok= True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded APLHAVANTAGE_API_KEY?", bool(ALPHA_KEY))

#print isi API nya
print(ALPHA_KEY)

Loaded APLHAVANTAGE_API_KEY? True
test_API_Azfina


In [32]:
#SCRAPE A SMALL TABLE

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Yahoo Finance Most Active Stocks page
url = "https://finance.yahoo.com/most-active"

# Request the page
headers = {"User-Agent": "Mozilla/5.0"}  # Pretend to be a browser
response = requests.get(url, headers=headers)
response.raise_for_status()

# Parse with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Find the table
table = soup.find("table")  # Yahoo's first table on the page

# Extract data
rows = []
for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cells:
        rows.append(cells)

# Assume first row is header
header, *data = rows

# Create DataFrame
df = pd.DataFrame(data, columns=header)

print(df.head())

#save to CSV in data/raw
df.to_csv("data/raw/api_yahoo_most_active_stocks_20250815_2030.csv", index=False)
 

 


  Symbol                        Name                    Price   Change  \
0   OPEN  Opendoor Technologies Inc.    3.1700+0.1300(+4.28%)  +0.1300   
1   INTC           Intel Corporation       24.56+0.70(+2.93%)    +0.70   
2   NVDA          NVIDIA Corporation      180.45-1.57(-0.86%)    -1.57   
3     NU            Nu Holdings Ltd.       13.10+1.09(+9.08%)    +1.09   
4   WULF               TeraWulf Inc.        8.97+0.26(+2.99%)    +0.26   

  Change %    Volume Avg Vol (3M) Market Cap P/E Ratio(TTM) 52 WkChange %  \
0   +4.28%  410.691M     194.061M     2.333B             --       +69.83%   
1   +2.93%  307.455M      87.982M   107.499B             --       +14.33%   
2   -0.86%  150.295M     183.388M     4.401T          58.40       +46.11%   
3   +9.08%  130.401M      61.716M      63.2B          29.77       -14.76%   
4   +2.99%  107.212M      50.714M     3.516B             --      +116.13%   

  52 Wk Range  
0              
1              
2              
3              
4           

In [33]:
#BEAUTIFUL SOUP

import requests
from bs4 import BeautifulSoup
import pandas as pd

# --- SETTINGS ---
symbol = "AAPL"  # Example: Apple
ALPHA_KEY = "YOUR_ALPHA_VANTAGE_API_KEY"

# --- TRY ALPHA VANTAGE FIRST ---
try:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={ALPHA_KEY}"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    js = r.json()

    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"

    series = js[key[0]]
    df_api = (
        pd.DataFrame(series).T
        .rename_axis("date")
        .reset_index()
        .rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        })
    )

    print("✅ Data from Alpha Vantage API")
    print(df_api.head())

except Exception as e:
    print("⚠ API failed, falling back to Yahoo Finance scrape:", e)

    # --- FALLBACK: SCRAPE YAHOO FINANCE ---
    scrape_url = f"https://finance.yahoo.com/quote/{symbol}/history"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    resp = requests.get(scrape_url, headers=headers, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
        if cells:
            rows.append(cells)
    
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

    print("✅ Data from Yahoo Finance scraping")
    print(df_scrape.head())


#save result to CSV in data/raw
df.to_csv("data/raw/scrape_alphavantage_most_active_stocks_20250815_2030", index=False)

✅ Data from Alpha Vantage API
         date      Open      High       Low     Close    Volume
0  2025-08-15  234.0000  234.2800  229.3350  231.5900  54908236
1  2025-08-14  234.0550  235.1200  230.8500  232.7800  51916275
2  2025-08-13  231.0700  235.0000  230.4300  233.3300  69878546
3  2025-08-12  228.0050  230.8000  227.0700  229.6500  55672301
4  2025-08-11  227.9200  229.5600  224.7600  227.1800  61806132
